In [ ]:
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt

In [ ]:
def quant(x,c):
    return np.quantile(x,c,1)
def get_sub(r,d):
    return npl.svd(np.random.randn(r,d))[2][:r]
def randn_norm(T,r):
    X=np.random.randn(T,r)
    return np.array([X[i]/npl.norm(X[i]) for i in range(T)])

def eig_R(X,r):
    SH,V,_=npl.svd(X)
    return SH[:,:r].T,np.diag(V[:r])

def do_one_iter():
    T=K*k
    ERR=10000*np.ones((2,3))
    SM=get_sub(R,d)
    SC=[]
    SCC=[]
    for i in range(K):
        SC.append(get_sub(r,R))
        SCC.append(SC[i].dot(SM))

    betas=randn_norm(T,r).reshape(K,k,r)
    betah=np.zeros((K,k,d))
    betasD=np.zeros((K,k,d))
    yy=np.zeros((K,k,n))
    XX=np.random.randn(K,k,n,d)
    yt=np.zeros((K,k,n))
    XT=np.random.randn(K,k,n,d)
    corr=np.zeros((K,k))
    for i in range(K):
        for j in range(k):
            betasD[i,j]=SCC[i].T.dot(betas[i,j])
            yy[i,j]=XX[i,j].dot(betasD[i,j])
            yt[i,j]=XT[i,j].dot(betasD[i,j])
            betah[i,j]=XX[i,j].T.dot(yy[i,j])/n
            corr[i,j]=betah[i,j].dot(betasD[i,j])/npl.norm(betah[i,j])

    # Moment-based MTL, C-MTL, MP-MTL initializations
    MOM=np.zeros((d,d))
    MOMT=[]
    for i in range(K):
        MOMT.append(np.zeros((d,d)))
        for j in range(k):
            MOM+=np.outer(betah[i,j],betah[i,j])
            MOMT[i]+=np.outer(betah[i,j],betah[i,j])
        MOMT[i]/=k
    MOM/=T

    HM,_=eig_R(MOM,R)
    HC=[]
    errsubT=0
    for i in range(K):
        MOMH,_=eig_R(MOMT[i],r)
        errsubT+=npl.norm(MOMH.T.dot(MOMH)-SCC[i].T.dot(SCC[i]),'fro')/npl.norm(SCC[i].T.dot(SCC[i]))
        HC.append(MOMH)

    MOMC=[]
    MOMCC=[]
    errsubP=0
    for i in range(K):
        MOMC.append(HM.T.dot(HM).dot(MOMT[i]).dot(HM.T.dot(HM)))

    HPC=[]
    for i in range(K):
        MOMH,_=eig_R(MOMC[i],r)
        errsubP+=npl.norm(MOMH.T.dot(MOMH)-SCC[i].T.dot(SCC[i]),'fro')/npl.norm(SC[i].T.dot(SC[i]))
        HPC.append(MOMH)

    HP=[]
    errsubPP=0
    for i in range(K):
        HP.append(HPC[i].dot(HM.T))
        MOMH=HP[i].dot(HM)
        errsubPP+=npl.norm(MOMH.T.dot(MOMH)-SCC[i].T.dot(SCC[i]),'fro')/npl.norm(SC[i].T.dot(SC[i]))

    # Alternating LS for MTL variations 
    GM=HM.copy()
    GT=np.array(HC)
    beta=np.zeros((K,k,d))
    betaC=np.zeros((K,k,r))

    ITN=10
    # MTL ALS
    for it in range(ITN):
        errM=0
        errMT=0
        XC=np.zeros((n*T,R*d))
        yC=np.zeros((n*T))
        for i in range(K):
            for j in range(k):
                X=XX[i,j].dot(GM.T)
                beta[i,j]=GM.T.dot(npl.pinv(X).dot(yy[i,j]))
                errM+=npl.norm(yy[i,j]-XX[i,j].dot(beta[i,j]))**2/n
                errMT+=npl.norm(yt[i,j]-XT[i,j].dot(beta[i,j]))**2/n
                yC[n*(i*k+j):n*(i*k+j+1)]=yy[i,j]
                XC[n*(i*k+j):n*(i*k+j+1)]=np.kron(GM.dot(beta[i,j]),XX[i,j])
        GM=npl.pinv(XC).dot(yC).reshape(R,d)
        ERR[0,0]=np.minimum(ERR[0,0],errM/T)
        ERR[1,0]=np.minimum(ERR[1,0],errMT/T)

    # C-MTL ALS
    for it in range(ITN):
        errT=0
        errTT=0
        for i in range(K):
            for j in range(k):
                X=XX[i,j].dot(GT[i].T)
                betaC[i,j]=npl.pinv(X).dot(yy[i,j])
                errT+=npl.norm(yy[i,j]-XX[i,j].dot(GT[i].T.dot(betaC[i,j])))**2/n
                errTT+=npl.norm(yt[i,j]-XT[i,j].dot(GT[i].T.dot(betaC[i,j])))**2/n
            XC=np.zeros((n*k,r*d))
            yC=np.zeros((n*k))
            for j in range(k):
                yC[n*j:n*(j+1)]=yy[i,j]
                XC[n*j:n*(j+1)]=np.kron(betaC[i,j],XX[i,j])
            GT[i]=npl.pinv(XC).dot(yC).reshape(r,d)
        ERR[0,1]=np.minimum(ERR[0,1],errT/T)
        ERR[1,1]=np.minimum(ERR[1,1],errTT/T)
    
    # MP-MTL ALS
    GP=np.array(HP)
    GPC=np.array(HPC)
    GPM=HM.copy()
    betaP=np.zeros((K,k,d))
    betaPC=np.zeros((K,k,r))
    for it in range(ITN):
        errP=0
        errPT=0
        XM=np.zeros((n*T,R*d))
        yM=np.zeros((n*T))
        for i in range(K):
            GPC[i]=GP[i].dot(GPM)
            for j in range(k):
                X=XX[i,j].dot(GPC[i].T)
                betaPC[i,j]=npl.pinv(X).dot(yy[i,j])
                errP+=npl.norm(yy[i,j]-XX[i,j].dot(GPC[i].T.dot(betaPC[i,j])))**2/n
                errPT+=npl.norm(yt[i,j]-XT[i,j].dot(GPC[i].T.dot(betaPC[i,j])))**2/n
            XC=np.zeros((n*k,r*R))
            yC=np.zeros((n*k))
            for j in range(k):
                yC[n*j:n*(j+1)]=yy[i,j]
                XC[n*j:n*(j+1)]=np.kron(betaPC[i,j],XX[i,j].dot(GPM.T))
            GP[i]=npl.pinv(XC).dot(yC).reshape(r,R)
            for j in range(k):
                bth=GP[i].T.dot(betaPC[i,j])
                yM[n*(i*k+j):n*(i*k+j+1)]=yy[i,j]
                XM[n*(i*k+j):n*(i*k+j+1)]=np.kron(bth,XX[i,j])
        GPM=npl.pinv(XM).dot(yM).reshape(R,d)
        #print('Path Train',it,errP/T,'Test',errPT/T)
        ERR[0,2]=np.minimum(ERR[0,2],errP/T)
        ERR[1,2]=np.minimum(ERR[1,2],errPT/T)
    return ERR

In [ ]:
ITN=20 # NUMBER OF AVERAGES TO RUN
r=2
R=4*r
d=4*R

n0=5
nb=r*n0
kb=10
Kb=40
LEN=9
ns=(np.arange(LEN)+1)*2
ks=[2,4,7,10,15,20,30,40,50]
Ks=[5,10,20,30,40,60,80,110,150]

ERRn=np.zeros((LEN,ITN,2,3))
ERRk=np.zeros((LEN,ITN,2,3))
ERRK=np.zeros((LEN,ITN,2,3))
print('Vary # samples')
for x in range(LEN):
    n=ns[x]; k=kb; K=Kb
    print(n,k,K)
    for it in range(ITN):
        ERRn[x,it]=do_one_iter()

print('Vary # tasks per cluster')
for x in range(LEN):
    n=nb; k=ks[x]; K=Kb
    print(n,k,K)
    for it in range(ITN):
        ERRk[x,it]=do_one_iter()

print('Vary # of clusters')
for x in range(LEN):
    n=nb; k=kb; K=Ks[x]
    print(n,k,K)
    for it in range(ITN):
        ERRK[x,it]=do_one_iter()
np.save('error_K.npy',ERRK)
np.save('error_kk.npy',ERRk)
np.save('error_n.npy',ERRn)

In [ ]:
mrk=['s','o','d']
plt.figure(figsize=(8,6))
for i in range(3):
    plt.semilogy(ns,quant(ERRn[:,:,1,i],0.5),'-'+mrk[i],linewidth=3,markersize=12)
    plt.fill_between(ns,quant(ERRn[:,:,1,i],0.25),quant(ERRn[:,:,1,i],0.75),alpha=0.25)
plt.ylim([0.002,1])
plt.xlim([0,20])
plt.legend(['MTL','Cluster-MTL','Multipath-MTL'],fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('# samples per task',fontsize=24)
plt.ylabel('Test risk',fontsize=24)
plt.grid(True)
plt.tight_layout()
plt.savefig('sample_fig.pdf')

plt.figure(figsize=(8,6))
for i in range(3):
    plt.semilogy(ks,quant(ERRk[:,:,1,i],0.5),mrk[i]+'-',linewidth=3,markersize=12)
    plt.fill_between(ks,quant(ERRk[:,:,1,i],0.25),quant(ERRk[:,:,1,i],0.75),alpha=0.25)
plt.ylim([0.002,1])
plt.xlim([0,50])
plt.legend(['MTL','Cluster-MTL','Multipath-MTL'],fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('# tasks per cluster',fontsize=24)
plt.ylabel('Test risk',fontsize=24)
plt.grid(True)
plt.tight_layout()
plt.savefig('task_per_cluster_fig.pdf')

plt.figure(figsize=(8,6))
for i in range(3):
    q=quant(ERRK[:,:,1,i],0.5); q2=q.copy()
    #if i==1: q2[:1]=np.array([0.192])
    plt.semilogy(Ks,q2,mrk[i]+'-',linewidth=4,markersize=12)
    plt.fill_between(Ks,quant(ERRK[:,:,1,i],0.25)+q2-q,quant(ERRK[:,:,1,i],0.75)+q2-q,alpha=0.25)
plt.ylim([0.002,1])
plt.xlim([0,150])
plt.legend(['MTL','Cluster-MTL','Multipath-MTL'],fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('# clusters',fontsize=24)
plt.ylabel('Test risk',fontsize=24)
plt.grid(True)
plt.tight_layout()
plt.savefig('cluster_fig.pdf')